In [2]:
import xarray as xr
import numpy as np

import skimage.measure as skm
import matplotlib.pyplot as plt
import cartopy

import os
home = os.path.expanduser("~")

import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [3]:
import sys
sys.path.insert(0, '/Users/cbla0002/Documents/code/phd/functions')

In [4]:
from myFuncs import *

In [5]:
onePlus(5)

6

In [ ]:
home = '/g/data/k10/cb4968'
home = '/Users/cbla0002/Documents/code/'
import sys
sys.path.insert(0, '{}phd/functions'.format(home))
from myFuncs import *